In [ ]:
import os

In [ ]:
def calcular_aciertos_metodo(carpeta_entrada, subcarpeta, metodo="gaussiano"):
    es_acierto = 0

    if metodo == "wavelet":
        archivo_info = "info_nitidez_wavelet.txt"
    else:
        archivo_info = f"info_varianzas_{metodo}.txt"

    ruta_archivo_info = os.path.join(carpeta_entrada, subcarpeta, metodo, archivo_info)

    if not os.path.isfile(ruta_archivo_info):
        print(f"El archivo {ruta_archivo_info} no existe.")
        return 0

    # Leer todas las líneas del archivo
    with open(ruta_archivo_info, "r") as archivo:
        lineas = archivo.readlines()

    # Filtrar las líneas relevantes (que contienen "->")
    lineas_relevantes = [linea for linea in lineas if "->" in linea]

    # Extraer los valores numéricos de las líneas relevantes
    valores = {}
    for linea in lineas_relevantes:
        partes = linea.split("->")
        if len(partes) == 2:
            nombre_imagen = partes[0].strip()
            valor = float(partes[1].split(":")[1].strip())
            valores[nombre_imagen] = valor

    # Verificar si el valor de la imagen "sharp.png" es el más alto
    sharp_key = subcarpeta + "_sharp.png"
    if sharp_key and valores[sharp_key] == max(valores.values()):
        es_acierto = 1

    return es_acierto

In [ ]:
def calcular_estadísticas():
    carpeta_entrada = "images_procesadas"
    carpeta_salida = "resultados_experimentacion/nitidez"
    archivo_salida = os.path.join(carpeta_salida, "estadisticas.txt")
    
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Inicializar contadores
    aciertos_gaussiano = 0
    aciertos_mediana = 0
    aciertos_media = 0
    aciertos_wavelet = 0
    total_imagenes = 0

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        aciertos_gaussiano += calcular_aciertos_metodo(carpeta_entrada, subcarpeta, "gaussiano")
        aciertos_mediana += calcular_aciertos_metodo(carpeta_entrada, subcarpeta, "mediana")
        aciertos_media += calcular_aciertos_metodo(carpeta_entrada, subcarpeta, "media")
        aciertos_wavelet += calcular_aciertos_metodo(carpeta_entrada, subcarpeta, "wavelet")
        total_imagenes += 1

    # Calcular porcentajes de aciertos
    if total_imagenes > 0:
        aciertos_gaussiano = (aciertos_gaussiano / total_imagenes) * 100
        aciertos_mediana = (aciertos_mediana / total_imagenes) * 100
        aciertos_media = (aciertos_media / total_imagenes) * 100
        aciertos_wavelet = (aciertos_wavelet / total_imagenes) * 100
    else:
        print("No se encontraron imágenes para evaluar.")
        return
    
    mejores_metodos = []

    mejor_resultado = max(aciertos_gaussiano, aciertos_mediana, aciertos_media, aciertos_wavelet)
    if mejor_resultado == aciertos_gaussiano:
        mejores_metodos.append("gaussiano")
    if mejor_resultado == aciertos_mediana:
        mejores_metodos.append("mediana")
    if mejor_resultado == aciertos_media:
        mejores_metodos.append("media")
    if mejor_resultado == aciertos_wavelet:
        mejores_metodos.append("wavelet")

    with open(archivo_salida, "w", encoding="utf-8") as archivo:
        archivo.write(f"Aciertos método gaussiano: {aciertos_gaussiano}%\n")
        archivo.write(f"Aciertos método mediana: {aciertos_mediana}%\n")
        archivo.write(f"Aciertos método media: {aciertos_media}%\n")
        archivo.write(f"Aciertos método wavelet: {aciertos_wavelet}%\n\n")

        if len(mejores_metodos) > 1:
            archivo.write("Los mejores métodos son:\n")
            for metodo in mejores_metodos:
                archivo.write(f"- {metodo}\n")
            archivo.write(f"Con {mejor_resultado}% de aciertos.\n")
        else:
            archivo.write(f"Mejor método: {mejores_metodos[0]} con {mejor_resultado}% de aciertos\n")
    
    print("Evaluación de los métodos de nitidez completada.")

In [ ]:
calcular_estadísticas()